In [18]:
import numpy as np
import time
import heapq
import math
import pandas as pd

# Hedef durumu tanımla
goal_state = tuple([1, 2, 3, 4, 5, 6, 7, 8, 0])

# Rastgele bir başlangıç durumu üretme fonksiyonu
def generate_random_initial_state():
    state = np.arange(9)
    np.random.shuffle(state)
    return tuple(state)

# Durum haritasını basit bir görsel olarak gösterme fonksiyonu
def format_state(state):
    state_array = np.array(state).reshape(3, 3)
    return '\n'.join([' '.join(map(str, row)) for row in state_array])

# Heuristik hesaplama fonksiyonları
def manhattan_distance(state):
    distance = 0
    for index, value in enumerate(state):
        if value != 0:
            target_x, target_y = divmod(value - 1, 3)
            current_x, current_y = divmod(index, 3)
            distance += abs(target_x - current_x) + abs(target_y - current_y)
    return distance

def misplaced_tile_count(state):
    return sum(1 for i, val in enumerate(state) if val != 0 and val != goal_state[i])

def sqrt_manhattan_distance(state):
    return math.sqrt(manhattan_distance(state))

def max_heuristic(state):
    max_dist = 0
    for index, value in enumerate(state):
        if value != 0:
            target_x, target_y = divmod(value - 1, 3)
            current_x, current_y = divmod(index, 3)
            dist = abs(target_x - current_x) + abs(target_y - current_y)
            max_dist = max(max_dist, dist)
    return max_dist

# Komşu durumları oluşturma
def generate_neighbors(state):
    neighbors = []
    state = list(state)
    zero_index = state.index(0)
    x, y = divmod(zero_index, 3)
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            neighbor = state[:]
            swap_index = nx * 3 + ny
            neighbor[zero_index], neighbor[swap_index] = neighbor[swap_index], neighbor[zero_index]
            neighbors.append(tuple(neighbor))
    return neighbors

# RBFS algoritması
def rbfs(state, heuristic, depth_limit=50):
    frontier = []
    heapq.heappush(frontier, (heuristic(state), state, 0))
    explored_nodes = 0
    start_time = time.time()
    visited = set()
    
    initial_map = format_state(state)  # İlk durum haritasını al

    while frontier:
        _, current_state, depth = heapq.heappop(frontier)
        
        if depth > depth_limit:
            return time.time() - start_time, explored_nodes, initial_map, format_state(goal_state)
        
        explored_nodes += 1
        visited.add(current_state)
        
        if current_state == goal_state:
            return time.time() - start_time, explored_nodes, initial_map, format_state(goal_state)

        for neighbor in generate_neighbors(current_state):
            if neighbor not in visited:
                cost = depth + 1
                priority = cost + heuristic(neighbor)
                heapq.heappush(frontier, (priority, neighbor, cost))

    return time.time() - start_time, explored_nodes, initial_map, format_state(goal_state)

# RBFS simülasyonu
def simulate_rbfs():
    initial_state = generate_random_initial_state()
    print("Başlangıç Durumu:\n" + format_state(initial_state))
    
    heuristics = [
        ("Manhattan Mesafesi", manhattan_distance),
        ("Hatalı Taş Sayısı", misplaced_tile_count),
        ("Manhattan Mesafesinin Karekökü", sqrt_manhattan_distance),
        ("Maksimum Heuristik", max_heuristic)
    ]
    results = []

    for name, heuristic in heuristics:
        time_taken, nodes_explored, initial_map, goal_map = rbfs(initial_state, heuristic)
        results.append({
            "Algoritma": name,
            "Başlangıç Durumu": initial_map,
            "Hedef Durumu": goal_map,
            "Çözüm Süresi (saniye)": round(time_taken, 4),
            "Keşfedilen Düğüm Sayısı": nodes_explored
        })
    
    # Tabloyu DataFrame olarak göster
    results_df = pd.DataFrame(results)
    return results_df

# Sonuçları göster
results_df = simulate_rbfs()
pd.set_option('display.max_colwidth', None)  # Tüm metni göster
styled_df = results_df.style.set_properties(**{'text-align': 'center'}) \
    .set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) \
    .set_caption("RBFS Algoritması ile 8 Taş Bulmacası Çözümü") \
    .format({"Çözüm Süresi (saniye)": "{:.4f}", "Keşfedilen Düğüm Sayısı": "{:,.0f}"})

styled_df


Başlangıç Durumu:
0 3 1
6 8 7
5 4 2


,Algoritma,Başlangıç Durumu,Hedef Durumu,Çözüm Süresi (saniye),Keşfedilen Düğüm Sayısı
0,Manhattan Mesafesi,0 3 1 6 8 7 5 4 2,1 2 3 4 5 6 7 8 0,4.3805,"289,286"
1,Hatalı Taş Sayısı,0 3 1 6 8 7 5 4 2,1 2 3 4 5 6 7 8 0,3.6546,"414,252"
2,Manhattan Mesafesinin Karekökü,0 3 1 6 8 7 5 4 2,1 2 3 4 5 6 7 8 0,7.3324,"500,881"
3,Maksimum Heuristik,0 3 1 6 8 7 5 4 2,1 2 3 4 5 6 7 8 0,7.4223,"470,628"
